## CS431/631 Data Intensive Distributed Computing
### Winter 2019 - Assignment 4
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _replace this with your name_
* **ID:** _replace this with your UW student ID number_

---
#### Overview
For this assignment, you will be using Python and Spark to perform some simple analyses on relational (tabular) data.  You will use Spark to read tabular data from files and then answer simple queries about the data in those tables.

In addition to Python and Spark, you will need to use a little bit of SQL.  If you are already familiar with SQL, great.  If not, you will want to spend a short amount of time getting familiar with the basics.   Type "SQL tutorial" into your favorite search engine, and you will find many examples of text, interactive and video tutorials.   As a simple starting point, you might also want to look at [these slides](https://cs.uwaterloo.ca/~kmsalem/courses/cs743/F14/slides/sql.pdf), which give a short introduction to SQL.   Even this is much more than you will need for this assignment.

You will be working with tabular data based on the schema for the TPC-H benchmark, which is a standard test used to measure the performance of relational database systems.   The schema defines the tables that exist in the database, the information in each table, and relationships between information in one table and information in another.   The TPC-H schema models business information:  customers, orders, parts, suppliers, and so on.   You can find a diagram illustrating the TPC-H schema in the lecture notes.   You can also find it on page 13 of the [TPC-H benchmark specification](http://www.tpc.org/TPC_Documents_Current_Versions/pdf/tpc-h_v2.17.3.pdf).   The TPC-H schema is important for this assignment, so make sure that you keep this schema handy.

Finally, for this assignment you will be using Spark in a slightly different way than you have used it so far.  For previous assignments, you have used the original Spark RDD interface.   For this assignment, you will be using the newer Spark interface, which is based on DataFrames.   DataFrames are RDDs in which each element is constrained to have the same structure.   You can think of a DataFrame like a table in a relational database.   Each element of the DataFrame is a row or record in the table.   All records have the same structure.   There is a programming guide for Spark DataFrames [here](https://spark.apache.org/docs/latest/sql-programming-guide.html).  Start with that. There is also a [more detailed guide to the full programming interface](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html).

---
#### Getting Started
To get started, let's initialize Spark, load a couple of the TPC-H tables, and run some simple queries.
First, as always, we need to tell the Python interpreter where to find Spark, so run `findspark.init`

In [ ]:
import findspark
findspark.init("/u/cs451/packages/spark")

Next, we launch Spark.  When you used the RDD interface for previous assignments, you created a `SparkContext` when you launched Spark.   To use Spark SQL and the DataFrame interface, you instead create a `SparkSession`.   You do that as shown in the next cell (run it!).    

In [ ]:
from pyspark.sql import SparkSession
import random
spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

Next, let's create DataFrames from the TPC-H data files.  We have installed the TPC-H data files in the directory `/u/cs451/data/TPC-H-0.1-TXT/`. 

If you are working on your own machine, you can download the dataset [here](https://roegiest.com/bigdata-2019w/assignments/TPC-H-0.1-TXT.tar.gz) and extract it to the same location on your system, for convenience. If you decide to modify the location of the dataset, you will have to change the path provided to `spark.read.csv()` for the examples that follow. You will be able to provide a different path when answering the questions.

There is one file for each table in the TPC-H database, e.g., `nation.tbl` for the TPC-H Nation table, `customer.tbl` for the TPC-H Customer table, and so on.    These are plain text csv files, with the character `|` used as a field separator.

Create a Spark DataFrame corresponding to the TPC-H Nation table by loading the data from the `nation.tbl` file.   Run the code in the next cell to do this.   After you run this code, `nation_raw` will refer to your new Spark DataFrame.   The Spark `show()` method will display a small (default 20) number of elements from the DataFrame, so that you can inspect them. 

In [ ]:
nation_raw = spark.read.csv("/u/cs451/data/TPC-H-0.1-TXT/nation.tbl",sep='|',inferSchema=True)
nation_raw.show()

Now you have a DataFrame to work with.   The columns of the DataFrame correspond to the fields of the TPC-H Nation table, so have a look at the TPC-H schema diagram to see what you are dealing with.   Column c0 is the NATIONKEY, column c1 is the NAME, c2 is the REGIONKEY, and so on.   Since this is a synthetic database, you'll notice that the data in some of the fields (like the COMMENT field) consists of random words.   That's fine.   You can also ask Spark to tell you about the type of data in each column:

In [ ]:
nation_raw.dtypes

Before going on, let's clean this DataFrame up a bit, to make it easier to use.   First, let's assign names to the columns, so that we can remember what information each column holds.   Second, you'll notice that Spark has created an extra final column (filled with `null` values) because each line in the input file ends with a separator character (|).  Let's drop that final column, since we don't need it.   Run the following code to do this:

In [ ]:
nation = nation_raw.toDF('NationKey','Name','RegionKey','Comment','extra').drop('extra').cache()
nation.show()

This style of code should look familar to you.  We started with the `nation_raw` DataFrame and applied a series of DataFrame operations (`toDF`, `drop`, and `cache`).   This is just like the RDD interface, except now we are applying DataFrame operations to DataFrames, instead of RDD operations to RDDs.

Next, let's load up the TPC-H Supplier table, and then try performing some queries:

In [ ]:
supplier_raw = spark.read.csv("/u/cs451/data/TPC-H-0.1-TXT/supplier.tbl",sep='|',inferSchema=True).drop("_c7")
supplier = supplier_raw.toDF("SuppKey","Name","Address","NationKey","Phone","AcctBal","Comment").cache()
supplier.show()

---
#### Writing Queries
There are two equivalent ways of writing queries over Spark DataFrames.   The first way is to assign a "view name" to the DataFrame, and then write SQL queries referring to those view names using the `sql` operation.  

The code below gives the view names "nation" and "supplier" to the two DataFrames we've already created.

In [ ]:
supplier.createOrReplaceTempView("supplier")
nation.createOrReplaceTempView("nation")

Now, we can write SQL queries that refer to the "supplier" and "nation" views as tables.   For example, suppose we want to see the names and addresses of suppliers who have account balances above 9900.00:

In [ ]:
q1_result = spark.sql("select Name, Address, AcctBal from supplier where AcctBal > 9900.00")
q1_result.show()

In the example above, the `sql` command runs the SQL query against the supplier table.   It returns the query result as a new DataFrame, which `q1_result` refers to.

Instead of writing your queries in SQL and running them using `sql`, it is possible to do the same thing by applying a sequence of DataFrame operations to the input DataFrames, as you did when you were using the RDD interface in the previous assignments.    For example, to answer the same query that we just answered using SQL, we can do the following:

In [ ]:
q1_resultB = supplier.filter("AcctBal > 9900.00").select('Name','Address','AcctBal')
q1_resultB.show()

Both methods should give the same result.   Internally, Spark handles both similarly.   For this assignment, you'll be asked to try out both methods.

Now it is time for you to try writing your own queries.

### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **not** change the prototypes of the functions. Do **not** write code outside of the functions. 

##### You may use specific cells, identified by `# Your tests here`, for test purposes. Code in these cells will *not* be executed when marking your assignment. 

##### We will make sure the function `load_dataset_and_set_views()` is called before your answers to other questions.

---
#### Question 0
First, you will instantiate the DataFrames and set the views needed to answer the following questions. 
Modify the code in the cell below to read the corresponding tables from disk in the same fashion as the supplier table. Keep the column names consistent with the [TPC-H schema](http://www.tpc.org/TPC_Documents_Current_Versions/pdf/tpc-h_v2.17.3.pdf) (page 13).

In [ ]:
def load_dataset_and_set_views(path="/u/cs451/data/TPC-H-0.1-TXT/"):
    global supplier, orders, customer, partsupp, nation, part
    
    supplier_raw = spark.read.csv(path+"supplier.tbl",sep='|',inferSchema=True).drop("_c7")
    supplier = supplier_raw.toDF("SuppKey","Name","Address","NationKey","Phone","AcctBal","Comment").cache()
    supplier.createOrReplaceTempView("supplier")
    
    # Your solution to Question 0 here
    order_raw = None
    orders = None
    orders.createOrReplaceTempView("orders")

    customer_raw = None
    customer = None
    customer.createOrReplaceTempView("customer")
    
    partsupp_raw = None
    partsupp = None
    partsupp.createOrReplaceTempView("partsupp")
    
    nation_raw = None
    nation = None
    nation.createOrReplaceTempView("nation")
    
    part_raw = None
    part = None
    part.createOrReplaceTempView("part")

In [ ]:
# Your tests here
load_dataset_and_set_views()


---
#### Question 1 (2/25 marks)
In the cell below, implement the function `five_highest_totalprice_orders_sql()` that will return the ORDERKEY, ORDERDATE, and TOTALPRICE of the five orders with the highest TOTALPRICE. Express your query in SQL, and use `sql` to execute it. The function must return a DataFrame with the corresponding schema `ORDERKEY, ORDERDATE, TOTALPRICE`. Use the view(s) or DataFrame(s) created in Question 0.

In [ ]:
def five_highest_totalprice_orders_sql():
    # Your solution to Question 1 here
    pass


In [ ]:
# Your tests here
five_highest_totalprice_orders_sql().show()

---
#### Question 2 (2/25 marks)
In the cell below, implement the function `five_highest_totalprice_orders_dtf()` to answer the same query you answered in Question 1, but this time do not use the `sql` method. The function must returns a DataFrame with the corresponding schema `ORDERKEY, ORDERDATE, TOTALPRICE`, ordered by TOTALPRICE value. Use the view(s) or DataFrame(s) created in Question 0.

In [ ]:
def five_highest_totalprice_orders_dtf():
    # Your solution to Question 2 here
    pass


In [ ]:
# Your tests here
five_highest_totalprice_orders_dtf().show(5)

---
#### Question 3 (3/25 marks)
In the cell below, implement the function `cust_most_recent_order_sql(custkey)` that takes a Customer Key as an input and returns the customer's name as well as the ORDERDATE and TOTALPRICE of that customer's most recent order.   Express the query in SQL, and use `sql` to execute it. You will need to use information from multiple tables to generate your answer. Use the view(s) or DataFrame(s) created in Question 0.

In [ ]:
def cust_most_recent_order_sql(custkey):
    # Your solution to Question 3 here
    pass


In [ ]:
# Your tests here
cust_most_recent_order_sql(0).show()


---
#### Question 4 (3/25 marks)
In the cell below, answer the same query you answered in Question 3, but this time do not use the `sql` method. Use the view(s) or DataFrame(s) created in Question 0.

In [ ]:
def cust_most_recent_order_dtf(custkey):
    # Your solution to Question 4 here
    pass


In [ ]:
# Your tests here
cust_most_recent_order_dtf(0).show(1)


---
#### Question 5 (5/25 marks)
In the cell below, implement `distinct_supplied_parts(nname)` that will return the number of distinct parts supplied by suppliers that are located in the given nation. The function must output an **integer** (not a DataFrame).

You may answer this question with or without using `sql` - whichever you prefer. Use the view(s) or DataFrame(s) created in Question 0.

In [ ]:
def distinct_supplied_parts(nname):
    # Your solution to Question 5 here
    pass


In [ ]:
# Your tests here
distinct_supplied_parts("CANADA")


---
#### Question 6 (5/25 marks)
In the cell below, implement `count_suppliers_brand_per_nation(bname)` that takes a brand name as input, like those that appear in the Parts table.  Given the brand,
report, for each nation, a count of that nation's suppliers of parts having that brand, ordered by nation name.   Your output should be a table (i.e. DataFrame) of nations and their supplier counts. Each supplier should be counted at most once in a nation's total, even if that supplier produces multiple parts with the given brand.

You may answer this question with or without using `sql` - whichever you prefer. Use the view(s) or DataFrame(s) created in Question 0.

In [ ]:
def count_suppliers_brand_per_nation(bname):
    # Your solution to Question 6 here
    pass


In [ ]:
# Your tests here
count_suppliers_brand_per_nation("Brand#14").show()

---
#### Question 7 (5/25 marks)
In the cell below, write code for the function `order_number_per_customer_nation(nname)` that will return, for a given nation name, for each year, the total number of orders placed by customers from the specified Nation, ordered by descending number of orders.

You may answer this question with or without using `sql` - whichever you prefer. Use the view(s) or DataFrame(s) created in Question 0.

In [ ]:
def order_number_per_customer_nation(nname):
    # Your solution to Question 7 here
    pass


In [ ]:
# Your tests here
order_number_per_customer_nation("CANADA").show()

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.

**Make sure that your notebook runs properly on the Jupyter hub before submitting it.**